**The code cells below is to download a dataset into Colab directly.** Skip if irrelevant

In [1]:
%%bash
# change url if you want to work with a different RSG dataset
wget -q --show-progress "https://russiansuperglue.com/tasks/download/RCB" -O temp.zip
unzip temp.zip -d data

# remove unnecessary directories and files
rm temp.zip
rm -r data/__MACOSX
rm -r sample_data/

Archive:  temp.zip
   creating: data/RCB/
  inflating: data/RCB/train.jsonl    
   creating: data/__MACOSX/
   creating: data/__MACOSX/RCB/
  inflating: data/__MACOSX/RCB/._train.jsonl  
  inflating: data/RCB/.DS_Store      
  inflating: data/__MACOSX/RCB/._.DS_Store  
  inflating: data/RCB/test.jsonl     
  inflating: data/__MACOSX/RCB/._test.jsonl  
  inflating: data/RCB/val.jsonl      
  inflating: data/__MACOSX/RCB/._val.jsonl  
  inflating: data/__MACOSX/._RCB     



     0K .......... .......... .......... .......... .......... 37%  369K 0s
    50K .......... .......... .......... .......... .......... 74%  370K 0s
   100K .......... .......... .......... ...                  100%  179M=0.3s

Fancy way to display content of the dataset directory

In [2]:
%%bash
# add tree package for fancy directory display
apt-get install tree

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (119 kB/s)
Selecting previously unselected package tree.
(Reading database ... 144793 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [3]:
# display the content of the folder
!tree

.
└── data
    └── RCB
        ├── test.jsonl
        ├── train.jsonl
        └── val.jsonl

2 directories, 3 files


### **Start playing here**

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000)

class JSONL_handler():
    """ opens a jsonl file and turns it into a necessary data structure """
    
    def __init__(self, path):
        self.path = path # path to jsonl file

    def to_pandas(self):
        """ get jsonl file content as a pandas DataFrame"""
        return pd.read_json(path_or_buf=self.path, lines=True)

In [5]:
handler = JSONL_handler('data/RCB/train.jsonl')
df = handler.to_pandas()

[Pandas cheat sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf?platform=hootsuite) for reference


In [6]:
df.head(6)

,premise,label,hypothesis,verb,negation,genre,idx,no_negation
0,"Сумма ущерба составила одну тысячу рублей. Уточняется, что на место происшествия выехала следственная группа, которая установила личность злоумышленника. Им оказался местный житель, ранее судимый за подобное правонарушение.",entailment,Ранее местный житель совершал подобное правонарушение.,судить,no_negation,kp,0,NaN
1,"Перебрасываясь словечками, они скользят глазами по моему городу. Как они смеют смотреть, будто что-то понимают?",contradiction,Они что-то понимают,смотреть,no_negation,fiction,1,NaN
2,"— Разве что, — сказала она, — мы хотим где-нибудь выпить кофе. — Мне кажется, — сказал он, — что мы хотим. На следующей Он успел встать на эскалатор, когда понял, что это “Таганская”, а не “Тульская”.",neutral,"Это “ Таганская ”, а не “ Тульская ”.",понять,no_negation,fiction,2,NaN
3,"Зима, наконец, показала свой характер.",contradiction,У зимы есть свой характер.,показать,no_negation,kp,3,NaN
4,"ГуманностьБогаподобнымсценариемнепредполагается. Но Его благость остается в неприкосновенности. Непросто жить в таком мире, но кто сказал, что это должно быть просто?",contradiction,Это должно быть просто.,сказать,no_negation,fiction,4,NaN
5,"Для мамы праздник был как обычный день. Она кормила людей. Весеннее небо обещало счастье, но мать не верила ему.",neutral,Небо обещает счастье.,верить,negation,fiction,5,NaN


In [7]:
df_contr = df[(df['label']== 'contradiction')]
df_contr.reset_index
df_contr.head()

,premise,label,hypothesis,verb,negation,genre,idx,no_negation
1,"Перебрасываясь словечками, они скользят глазами по моему городу. Как они смеют смотреть, будто что-то понимают?",contradiction,Они что-то понимают,смотреть,no_negation,fiction,1,NaN
3,"Зима, наконец, показала свой характер.",contradiction,У зимы есть свой характер.,показать,no_negation,kp,3,NaN
4,"ГуманностьБогаподобнымсценариемнепредполагается. Но Его благость остается в неприкосновенности. Непросто жить в таком мире, но кто сказал, что это должно быть просто?",contradiction,Это должно быть просто.,сказать,no_negation,fiction,4,NaN
9,"Обеспокоилась судьбой серебряных ложек, которые лежали у рабочих, как говорится, на виду. Вдруг украдут! Стала убирать. В ответ мама со словами: “Да как ты смеешь так думать, дрянь! Это же РАБОЧИЕ, понимаешь, РАБОЧИЕ!!",contradiction,Рабочие украдут ложки.,думать,no_negation,fiction,9,NaN
14,Вопрос качества работы орловских библиотек на днях обсудили в Правительстве Орловской области.,contradiction,Орловские библиотеки работают качественно.,обсудить,no_negation,kp,14,NaN


Выражения, которые могут служить индикаторами лэйбла contradiction: будто, ?, вдруг, вопрос, подозрева(е|ю)т, требовал, показал(а|и)сь, кажется/казаться, хотим, не, возможно, смеешь, хотя, разве, предполагал(а)?, думае(шь|те), считали, подумали, обман(ываться)*, нужно, а, заблуждаться, но 

Странные сегменты 162, 164
Не понятно, как рассматривать метафоры.
Сегменты 181, 186, 223, 337, 425 амбивалентны. Они одновременно и подходят, и не подходят к тексту.

In [8]:
#272, 280, 290, 303, 442 - возможно все же не contradiction
df.iloc[[27, 34, 66, 67, 68, 105, 138, 139, 187, 236, 243, 273, 302, 395]]

,premise,label,hypothesis,verb,negation,genre,idx,no_negation
27,"Не гневается на тех, кто, не останавливаясь, проходит мимо. Но сам он остро, пронзительно сострадает несчастному. И — заставляет задуматься, осознать, как хрупка и уязвима наша душа, как неповторимо единственна человеческая жизнь.",contradiction,"Хрупка и уязвима наша душа, неповторимо единственна человеческая жизнь.",осознать,no_negation,fiction,27,NaN
34,"Как я матерюсь, так она врет. Ловлена с поличным неоднократно, принципов своих не предавала, продолжала врать и обижаться, что ей не верят. Свято помня отцовский завет “Наказывают не за преступление, а за то, что попался”, никогда ни в чем не признается.",contradiction,Ей не верят.,обижаться,no_negation,fiction,34,NaN
66,"Тогда как собственная смерть не значит ничего. И вы благодарны вашему убийце за то, что он не брезгует подойти вплотную. Все так изложено - я бы сказал, даже чересчур ясно.",contradiction,Это изложено даже чересчур ясно.,сказать,no_negation,fiction,66,NaN
67,"Все вернулось на круги своя. Но во мне многое изменилось. Теперь я спокойно, даже отстраненно оглядывалась на себя прежнюю и не понимала, зачем так долго обманывала себя.",contradiction,Я долго обманывала себя.,понимать,negation,fiction,67,NaN
68,"Но этого хватило, чтобы скульптура получилась внушительного размера. Длина палубы крейсера - шесть метров. Не забыли и раскрасить для большего сходства с оригиналом.",contradiction,Скульптура была раскрашена.,забыть,negation,kp,68,NaN
105,Настоящий друг. Выручил меня с оркестром. Не забуду.,contradiction,Друг меня выручил.,забыть,negation,fiction,105,NaN
138,"Воспитывать, дорогие мои, надо на подвиге, на героизме. А этого бы хоть подстригли. Слышите, товарищ, я вам говорю, как там ваша фамилия,— неприлично мужчине ходить с такими волосами.",contradiction,Неприлично мужчине ходить с такими волосами.,говорить,no_negation,fiction,138,NaN
139,Дополнительные меры безопасности будут приняты на время новогодних и рождественских праздников.,contradiction,Во время новогодних и рождественских праздников будет безопаснее.,принять,no_negation,kp,139,NaN
187,"Мужчина пытался сбежать с места и даже кинулся на сотрудников полиции с желанием помахать кулаками. Агрессивного нарушителя скрутили, посадили в машину и отвезли в ближайший участок. Проверка на трезвость показала, что водитель был в состоянии алкогольного опьянения.",contradiction,Водитель был в состоянии алкогольного опьянения.,показать,no_negation,kp,187,NaN
236,"Наконец раздался голос. Мужской и смутно знакомый. Голос, запинаясь, медленно, как бы сомневаясь, что его услышат и поймут, произносил: – Добрый день.",contradiction,Его услышат.,сомневаться,no_negation,fiction,236,NaN


In [ ]:
df['label'].value_counts()

True     1061
False     688
Name: label, dtype: int64